In [1]:
!pip install nltk
!pip install spacy
!python -m spacy download pt_core_news_sm

2023-08-19 23:03:59.872617: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-19 23:04:01.413368: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 30.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [3]:
import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score

import string

In [10]:
df = pd.read_csv('./olist.csv')
df2 = pd.read_csv('./b2w.csv')
df = df.drop(["review_text", "review_text_tokenized", "polarity", "kfold_rating", "kfold_polarity", "original_index"], axis=1)
df2 = df2.drop(["review_text", "review_text_tokenized", "polarity", "kfold_rating", "kfold_polarity", "original_index"], axis=1)
df = df[:16000]
df2 = df2[:16000]
df.head()

,review_text_processed,rating
0,perfeito....chegou antes do prazo.....,5
1,foi uma otima compra! chegou antes mesmo do pr...,5
2,recebi muito rapido e um otimo custo beneficio,5
3,recomendo,5
4,so veio uma capa comprei 3 ai paguei. mais de ...,1


In [11]:
df2

,review_text_processed,rating
0,bem macio e felpudo...recomendo. preco imbati...,4
1,produto excepcional! recomendo!!! inovador e ...,5
2,recebi o produto antes do prazo mas veio com d...,1
3,bom custo beneficio. adequado para pessoas que...,5
4,alem de higienico tem o tamanho ideal. so falt...,3
...,...,...
15995,essa versao inclui a dlc? do ssj blue goku/ve...,5
15996,"super indico! material resistente, atendeu de ...",5
15997,quando realizei a compra o site mostrava que t...,1
15998,produto otimo recomendo a todos a bateria dura...,4


In [12]:
df["rating"] = np.where(df["rating"] <= 3, 0, 1)
df2["rating"] = np.where(df2["rating"] <= 3, 0, 1)

In [13]:
df = df.dropna()
df2 = df2.dropna()

In [18]:
personal_text = [
    "Local ruim e feio, cheira mal",
    "Local ruim, feio e sujo",
    "Esse lugar é um verdadeiro desastre - feio, sujo e com um cheiro insuportável.",
    "Nunca vi um local tão horrível em toda a minha vida - feio, malcuidado e fedorento.",
    "Esse é o tipo de lugar que você entra e já sente vontade de sair correndo - é feio, sujo e o cheiro é simplesmente repugnante.",
    "Não há como ignorar o fato de que esse local é uma combinação terrível de feiura, sujeira e mau odor.",
    "É impressionante como um lugar pode ser tão ruim em todos os sentidos - feio, sujo e com um cheiro que parece não ter fim.",
    "Não dá para suportar ficar aqui por muito tempo - é um lugar feio, sujo e com um fedor que gruda na pele.",
    "Parece que todas as coisas ruins se reuniram nesse lugar - é feio, sujo e o cheiro é simplesmente insuportável.",
    "Não entendo como alguém pode frequentar esse local - é feio, sujo e o cheiro deixa qualquer um enjoado.",
    "Esse lugar parece ter sido esquecido pelo resto do mundo - é feio, sujo e o cheiro é quase inacreditável.",
    "Não há como dar um passo sequer aqui sem ser atingido pela combinação de feiura, sujeira e um odor nauseante.",
    "A decadência deste lugar é tão evidente que é impossível não notar - é feio, sujo e o cheiro só piora a situação.",
    "Cada canto desse local é uma prova viva da falta de cuidado - é feio, sujo e o cheiro é simplesmente intolerável.",
    "Este lugar é como um pesadelo sensorial - feio, sujo e o cheiro é uma afronta ao olfato.",
    "A aparência repulsiva deste lugar é apenas a ponta do iceberg - a sujeira e o cheiro são igualmente perturbadores.",
    "A experiência de estar aqui é um desfile constante de desagrados - a feiura, a sujeira e o cheiro formam uma tríade terrível.",
    "Uma experiência desagradável em todos os sentidos.",
    "Este lugar é um desastre total.",
    "Evite a todo custo - é horrível aqui.",
    "Um local que não vale a pena visitar.",
    "Desanimadoramente feio e sujo.",
    "Uma mistura de elementos indesejáveis.",
    "Nada aqui parece agradável.",
    "Um local lamentável e deprimente.",
    "Cada canto é uma decepção.",
    "Uma experiência que quero esquecer.",
    "Evite, a menos que goste de desgosto.",
    "Um verdadeiro pesadelo para os sentidos.",
    "Pouco convidativo e desagradável.",
    "Aqui, tudo parece errado.",
    "Este lugar é um desgosto completo.",
    "Um local que deixa muito a desejar.",
    "Sensações negativas em cada visita.",
    "Nada aqui é agradável ou positivo."
]

personal_sentiment_negative = [0 for i in range(len(personal_text))]

In [22]:
positive_short_opinions = [
    "Adoro a atmosfera única deste lugar!",
    "Encanto escondido em cada canto.",
    "Uma experiência autêntica e memorável.",
    "Cada visita aqui é uma aventura!",
    "Um lugar que cativa a imaginação.",
    "Descobertas surpreendentes a cada visita.",
    "Aqui, beleza encontra singularidade.",
    "Um local cheio de charme e personalidade.",
    "Uma surpresa agradável em todos os sentidos.",
    "Encantadoramente imperfeito e irresistível.",
    "Sempre há algo novo para explorar aqui.",
    "Este lugar é uma pérola escondida.",
    "Uma mistura fascinante de elementos.",
    "Aqui, encontrei uma autenticidade rara.",
    "Beleza nas pequenas peculiaridades.",
    "Este lugar é cheio de surpresas positivas.",
    "Um local que tem sua própria magia.",
    "Experiências memoráveis a cada visita.",
    "Apesar dos desafios, este lugar tem um charme peculiar que o torna único.",
    "Aqui, mesmo com suas imperfeições, encontrei um certo encanto que é difícil de ignorar.",
    "A autenticidade deste local é algo que o torna especial, mesmo com suas características menos agradáveis.",
    "É interessante como, por trás das aparências, esse lugar possui sua própria singularidade e caráter.",
    "Apesar das aparências, há algo cativante nas entrelinhas deste local.",
    "Aqui, a mistura de elementos cria uma atmosfera intrigante que merece ser explorada.",
    "Há algo curiosamente atraente sobre a desordem e o caos deste lugar.",
    "Encontrei beleza nas imperfeições deste local, uma beleza que não é óbvia à primeira vista.",
    "Aqui, a autenticidade supera as preocupações estéticas, criando uma experiência genuína.",
    "Este lugar possui uma energia peculiar que transcende suas características visuais.",
    "A atmosfera deste local, embora não convencional, tem uma maneira de se tornar memorável.",
    "A singularidade deste lugar está em sua capacidade de se destacar da norma e deixar uma impressão duradoura.",
    "Apesar das primeiras impressões, existe um encanto subestimado que emerge ao explorar este local.",
    "Aqui, as peculiaridades se combinam para formar um mosaico encantador de experiências.",
    "Este é um lugar que desafia as expectativas e surpreende a cada descoberta.",
    "Há uma certa atmosfera de mistério neste local que o torna intrigante e interessante.",
    "Ao olhar além das aparências, este lugar revela uma riqueza de detalhes e histórias.",
    "Aqui, encontrei um cenário que, mesmo com suas falhas, possui um apelo cativante."
]

personal_sentiment_positive = [0 for i in range(len(positive_short_opinions))]

In [23]:
texts = personal_text + positive_short_opinions + list(df["review_text_processed"].values) + list(df2["review_text_processed"].values)
resp = personal_sentiment_negative + personal_sentiment_positive + list(df["rating"].values) + list(df2["rating"].values)

df = pd.DataFrame({
    "review_text_processed": texts,
    "rating": resp
})

In [24]:
df.shape

(32071, 2)

In [25]:
df.isnull().sum()

review_text_processed    0
rating                   0
dtype: int64

In [26]:
df = df.dropna()

In [27]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [28]:
stop_words = nltk.corpus.stopwords.words('portuguese')

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32071 entries, 0 to 32070
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   review_text_processed  32071 non-null  object
 1   rating                 32071 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 501.2+ KB


In [30]:
tweets = df["review_text_processed"]
tokenization = [word_tokenize(text.lower()) for text in tweets]

In [31]:
lemmatizer = WordNetLemmatizer()

new_tweets = []

for phrase in tokenization:
  new_phrase = ""
  for token in phrase:
    if not str(token) in stop_words and not token in string.punctuation and "@" not in token and "http" not in token and len(token) > 1 and not token.isdigit():
      new_phrase += lemmatizer.lemmatize(str(token)) + " "
  new_tweets.append(new_phrase[:-1])

In [32]:
df["review_text_processed"] = new_tweets

In [33]:
vect_uni_idf = TfidfVectorizer(ngram_range=(1,1), use_idf=True, norm='l2', stop_words=stop_words)
text_vect_uni_idf = vect_uni_idf.fit_transform(df["review_text_processed"])

X_trainUIDF, X_testUIDF, y_trainUIDF, y_testUIDF = train_test_split(text_vect_uni_idf, df["rating"], test_size=0.2, random_state=42)

In [34]:
rfcidf = RandomForestClassifier()

rfcidf.fit(X_trainUIDF, y_trainUIDF)
y_predUIDF = rfcidf.predict(X_testUIDF)

In [35]:
print(accuracy_score(y_testUIDF, y_predUIDF))

0.8656274356975838


In [36]:
rfcidf.predict(vect_uni_idf.transform(["local ruim, feio e sujo com cheiro de peixe"]))

array([0])

In [37]:
rfcidf.predict(vect_uni_idf.transform(["Praça muito bonita, porem tem bancos quebrados e ruins, chão muito sujo e brinquedos quebrados"]))

array([0])

In [ ]:
dtc = DecisionTreeClassifier()

dtc.fit(X_trainUIDF, y_trainUIDF)
y_preddtc = dtc.predict(X_testUIDF)

print(accuracy_score(y_testUIDF, y_preddtc))

0.8090262002120249


In [38]:
import joblib
model_filename = 'sentiment_analyser_model.pkl'
joblib.dump(rfcidf, model_filename)

['sentiment_analyser_model.pkl']

In [39]:
model_filename = 'sentiment_analyser_vect.pkl'
joblib.dump(vect_uni_idf, model_filename)

['sentiment_analyser_vect.pkl']